In [1]:
import gym
import numpy as np
import tensorflow as tf 
from keras.preprocessing.image import img_to_array
from keras.layers import Dense,Conv2D,Flatten
from keras.models import Sequential
from keras.optimizers import RMSprop
from collections import deque
import matplotlib.pyplot as plt
from keras.applications import imagenet_utils

/home/nijatmursali/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nijatmursali/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nijatmursali/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nijatmursali/.local/lib/python3.5/site-packages/tensorflow/python/f

In [2]:
learningRate = 0.1 #learning_rate
discountFactor = 0.95 #discount
numberofEpisodes = 10000 #episodes
howmanyepisodestoshow = 500 #show every
numberofbuckets = 10 #buckets
gridsize = [10, 10]
decayingstartepsilon = 1
decayingendepsilon = numberofEpisodes//2


In [3]:
def valueforDiscreteState(state): 
    discrete_state = (state - env.observation_space.low)/numberofbuckets
    return tuple(discrete_state.astype(np.int))  

def startLearning(qValue, render, needtoUpdateWeights):
    done = False
    discrete_state = valueforDiscreteState(env.reset()) #puts the car in random location in the field
    success = False

    while not done:
        
        #if the random value is more than epsilon
        if np.random.random() > np.finfo(float).eps: 
            #we need to use q-table to take current best action
            action = np.argmax(qValue[discrete_state])
        else:
            # or we need to explore the environment
            action = np.random.randint(0, env.action_space.n)

        # Run simulation step
        new_state, reward, done, _ = env.step(action)

        discretevalueforNewState = valueforDiscreteState(new_state)


        if new_state[0] >= env.goal_position:
            success = True

        #if we need to update, we update the Q value using numpy
        if needtoUpdateWeights:
            max_future_q = np.max(qValue[discretevalueforNewState])
            current_q = qValue[discrete_state + (action,)]
            newQValue = (1 - learningRate) * current_q + learningRate * (reward + discountFactor * max_future_q) #Q learning
            qValue[discrete_state + (action,)] = newQValue

        discrete_state = discretevalueforNewState

        if render:
            env.render()

    return success



In [4]:
env = gym.make("MountainCar-v0")

epsilon = 1  
env_high = env.observation_space.high
env_low = env.observation_space.low
changeforepsilon = epsilon/(decayingendepsilon - decayingstartepsilon)
numberofbuckets = (env_high - env_low)/gridsize
qValue = np.random.uniform(low=-3, high=0, size=(gridsize + [env.action_space.n]))
success = False

env.close()


In [5]:
def plottingValues(name):
    gettingDatafromFile = open("MountainCar-v0.txt","r").read()
    iterationAppend = []
    rewardAppend = []
    myData  =  gettingDatafromFile.split("\n")
    for data in myData:
        try:
            valueSplitting  = data.split(",")
            valuereward  =  valueSplitting[1]
            valueiteration = valueSplitting[0]

            
            iterationAppend.append(float(valueiteration))
            rewardAppend.append(float(valuereward))
            
            
        except:
            pass 
    
    nameforImage="MountainCar-v0.png"
    
    plt.plot(iterationAppend,rewardAppend)
    plt.xlabel("Episodes")
    plt.ylabel("Reward")
    plt.title("Reward and Episode Diagram")
    plt.savefig(nameforImage, bbox_inches="tight")
    plt.close()

In [6]:
episode = 0
numberofsuccesses = 0

while episode<numberofEpisodes:
    episode+=1
    done = False

    if episode % howmanyepisodestoshow == 0:
        print("Current episode: {}, success: {}, ({})".format(episode, numberofsuccesses, float(numberofsuccesses)/howmanyepisodestoshow))

        success = startLearning(qValue, True, False)
        numberofsuccesses = 0
    else:
        success = startLearning(qValue, False, True)

    if success:
        numberofsuccesses += 1

    # Move epsilon towards its ending value, if it still needs to move
    if decayingendepsilon >= episode >= decayingstartepsilon:
        epsilon -= changeforepsilon
        
    if episode%10 == 0: 
        openTxtFile  = open("MountainCar-v0.txt",'a')
        openTxtFile.write(str(episode)+","+str(numberofsuccesses))
        openTxtFile.write("\n")
        plottingValues("MountainCar-v0")

Current episode: 500, success: 30, (0.06)
Current episode: 1000, success: 102, (0.204)
Current episode: 1500, success: 199, (0.398)
Current episode: 2000, success: 263, (0.526)
Current episode: 2500, success: 327, (0.654)
Current episode: 3000, success: 284, (0.568)
Current episode: 3500, success: 370, (0.74)
Current episode: 4000, success: 167, (0.334)
Current episode: 4500, success: 178, (0.356)
Current episode: 5000, success: 327, (0.654)
Current episode: 5500, success: 287, (0.574)
Current episode: 6000, success: 439, (0.878)
Current episode: 6500, success: 490, (0.98)
Current episode: 7000, success: 469, (0.938)
Current episode: 7500, success: 500, (1.0)
Current episode: 8000, success: 500, (1.0)
Current episode: 8500, success: 475, (0.95)
Current episode: 9000, success: 313, (0.626)
Current episode: 9500, success: 352, (0.704)
Current episode: 10000, success: 348, (0.696)
